# SpaceRocks

### Vectorized coordinate transformation and ephemeris calculation with robust unit handling.

To install, simply `pip install spacerocks`

In [1]:
from spacerocks import Units, SpaceRock, Observe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u
%matplotlib inline

You can also pass in just a single object or an array of objects. I'll try all of the TNOs reported to the MPC. I specified an observatory code, so a topocentric correction will be applied to the Earth's position.

### Analyzing all of the TNOs in the MPC

In [2]:
TNOs = pd.read_json('https://minorplanetcenter.net/Extended_Files/distant_extended.json.gz')
TNOs = TNOs[TNOs.a * (1 - TNOs.e) > 30]
ETNOs = TNOs[TNOs.a > 230]
#TNOs = TNOs[TNOs.Principal_desig.values.astype(str) == '2015 BP519']
units = Units()
rocks = SpaceRock(a=TNOs.a.values, 
                  e=TNOs.e.values, 
                  inc=TNOs.i.values, 
                  arg=TNOs.Peri.values, 
                  node=TNOs.Node.values, 
                  t_peri=TNOs.Tp.values, 
                  epoch=TNOs.Epoch.values,
                  H=TNOs.H.values, 
                  name=TNOs.Principal_desig.values,
                  #delta_H = np.random.rand(len(TNOs)),
                  #rotation_period = np.random.uniform(0.2, 0.5, len(TNOs)),
                  #phi0 = np.random.rand(len(TNOs)) * 2 * np.pi,
                  input_frame='heliocentric',
                  units=units)

erocks = SpaceRock(a=ETNOs.a.values, 
                   e=ETNOs.e.values, 
                   inc=ETNOs.i.values, 
                   arg=ETNOs.Peri.values, 
                   node=ETNOs.Node.values, 
                   t_peri=ETNOs.Tp.values, 
                   epoch=ETNOs.Epoch.values,
                   H=ETNOs.H.values, 
                   name=ETNOs.Principal_desig.values,
                   #delta_H = np.random.rand(len(TNOs)),
                   #rotation_period = np.random.uniform(0.2, 0.5, len(TNOs)),
                   #phi0 = np.random.rand(len(TNOs)) * 2 * np.pi,
                   input_frame='heliocentric',
                   units=units)

#p = Propagate(rocks, np.linspace(2378480.5, 2378480.5 + 10, 10), model=1)

#obs_decam = Observe(p, obscode='W84', NSIDE=[128])
#obs_magellan = Observe(p, obscode=304, NSIDE=[256])
#obs_geocenter = Observe(p)

In [3]:
from astroquery.jplhorizons import Horizons

In [4]:
Nyears = 1
startdate = Time('2017-01-01', scale='utc', format='iso')
testdates = Time(np.arange(startdate.jd, startdate.jd + Nyears*365.25, 1), scale='utc', format='jd' )
tno_id = 'Ceres'
TNO_Horizons = Horizons(id=tno_id)
elements = TNO_Horizons.elements()[0]
ephem_Horizons = Horizons(id=tno_id, location='W84',
                          epochs={'start':testdates[0].iso, 'stop':testdates[-1].iso, 'step':'1d'}).ephemerides()
TNO = SpaceRock(a=elements['a'],
                e=elements['e'],
                inc=elements['incl'],
                arg=elements['w'],
                node=elements['Omega'],
                t_peri=elements['Tp_jd'],
                epoch=elements['datetime_jd'],
                name=[tno_id],
                input_frame='heliocentric',
                units=units)
TNO_prop = TNO.propagate(obsdates=testdates.jd, model=5)
TNO_predict = Observe(TNO_prop, obscode='W84')
pos_Horizons = SkyCoord(ephem_Horizons['RA'], ephem_Horizons['DEC'], frame='icrs', unit=(u.deg, u.deg))
pos_pred = SkyCoord(TNO_predict.ra.deg, TNO_predict.dec.deg, frame='icrs', unit=(u.deg, u.deg))
sep = pos_pred.separation(pos_Horizons)

UnitTypeError: Can only apply 'sin' function to quantities with angle units

In [ ]:
elem = Horizons(id=tno_id, epochs={'start':testdates[0].iso, 'stop':testdates[-1].iso, 'step':'1d'}).elements()

In [ ]:
TNO_prop.epoch[0]

In [ ]:
elem['a'][0]

In [ ]:
print(elem['datetime_jd'][0], TNO_prop.epoch[0])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['a'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.a, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('a', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['incl'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.inc.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['e'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.e, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, (elem['incl'] - TNO_prop.inc.deg), color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, (elem['a'] - TNO_prop.a), color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('a residual', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 12))
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['RA_rate'], color='black', s=50)
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.ra_rate.to(u.arcsec/u.h), color='red', s=10)
ax[0].set_xlabel('JD', fontsize=14)
ax[0].set_ylabel('ra rate (arcsec/h)', fontsize=14)
ax[0].set_xlim([0, Nyears]);

ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['DEC_rate'], color='black', s=50)
ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.dec_rate.to(u.arcsec/u.h), color='red', s=10)
ax[1].set_xlabel('JD', fontsize=14)
ax[1].set_ylabel('dec rate (arcsec/h)', fontsize=14)
ax[1].set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 12))
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, 
              ephem_Horizons['RA_rate'] - TNO_predict.ra_rate.to(u.arcsec/u.h), color='black', s=1)
ax[0].set_xlabel('JD', fontsize=14)
ax[0].set_ylabel('ra rate (arcsec/h)', fontsize=14)
ax[0].set_xlim([0, Nyears]);

ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, 
              ephem_Horizons['DEC_rate'] - TNO_predict.dec_rate.to(u.arcsec/u.h), color='black', s=1)
ax[1].set_xlabel('JD', fontsize=14)
ax[1].set_ylabel('dec rate (arcsec/h)', fontsize=14)
ax[1].set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, sep.arcsec, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Diff Horizons - SpaceRocks', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['elong'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.elong.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Solar Elongation Angle', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           ephem_Horizons['elong'] - TNO_predict.elong.deg, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Solar Elongation Angle Residual (deg)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['delta'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.delta, color='red', s=10)

ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Earth Distance', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))

ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (TNO_predict.delta - ephem_Horizons['delta']).to(u.km), color='black', s=1)

ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Earth Distance Residuals (km)', fontsize=14)
ax.set_xlim([0, Nyears]);
#ax.set_ylim([0, 200]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['RA'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.ra.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('ra', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (ephem_Horizons['RA'] - TNO_predict.ra.deg) * 3600, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('ra residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['DEC'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.dec.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('dec', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (ephem_Horizons['DEC'] - TNO_predict.dec.deg) * 3600, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('dec residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

---